# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../output_data/clean_city_data.csv")

# delete extra index Unnamed: 0
del city_data['Unnamed: 0']

city_data

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed
0,Andenes,NO,69.31,16.12,59.00,77,75,2.24
1,Dukat,RU,62.55,155.55,48.63,79,0,2.55
2,Hilo,US,19.73,-155.09,70.70,73,90,3.36
3,Busselton,AU,-33.65,115.33,51.71,91,94,9.22
4,Shache,CN,38.42,77.24,90.90,14,97,8.77
...,...,...,...,...,...,...,...,...
538,Solnechnyy,RU,50.72,136.63,69.22,85,100,1.66
539,Honningsvåg,NO,70.98,25.97,50.00,93,90,9.17
540,Paris,FR,48.85,2.35,75.52,49,20,12.75
541,Villaviciosa,ES,43.48,-5.44,77.00,58,0,7.87


### Humidity Heatmap
* Configure gmaps.

In [3]:
gmaps.configure(api_key=g_key)

* Use the Lat and Lng as locations and Humidity as the weight.

In [4]:
locations = city_data[["Latitude", "Longitude"]]
weight = city_data["Humidity"]

In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '989px',
    'height': '388px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(46.0, -5.0), zoom_level=2)

* Add Heatmap layer to map.

In [6]:
heat_layer = gmaps.heatmap_layer(locations, weight, dissipating=False, max_intensity=np.max(weight), point_radius=3)

In [7]:
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='388px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
hotel_df = city_data.loc[(city_data["Wind Speed"] <= 15) & (city_data["Cloudiness"] <= 10) &(city_data["Temp"] >= 75) & (city_data["Temp"] <= 95) & (city_data["Humidity"] <= 70)].dropna()

hotel_df

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed
5,Vila,PT,42.03,-8.16,84.58,45,0,3.36
22,Blagoveshchenka,RU,52.83,79.87,82.83,46,0,8.59
27,East London,ZA,-33.02,27.91,77.00,29,0,5.82
33,Mossamedes,AO,-15.20,12.15,81.73,29,1,11.45
56,Onega,RU,63.91,38.14,76.10,37,0,2.33
66,Saint George,US,37.10,-113.58,79.25,19,1,5.82
89,Samfya,ZM,-11.36,29.56,76.86,29,0,4.27
109,Richards Bay,ZA,-28.78,32.04,76.59,36,0,11.50
146,Luanda,AO,-8.84,13.23,75.42,66,1,14.05
149,Nokaneng,BW,-19.67,22.27,79.16,17,0,10.16


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.

In [9]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
5,Vila,PT,42.03,-8.16,84.58,45,0,3.36,
22,Blagoveshchenka,RU,52.83,79.87,82.83,46,0,8.59,
27,East London,ZA,-33.02,27.91,77.00,29,0,5.82,
33,Mossamedes,AO,-15.20,12.15,81.73,29,1,11.45,
56,Onega,RU,63.91,38.14,76.10,37,0,2.33,
66,Saint George,US,37.10,-113.58,79.25,19,1,5.82,
89,Samfya,ZM,-11.36,29.56,76.86,29,0,4.27,
109,Richards Bay,ZA,-28.78,32.04,76.59,36,0,11.50,
146,Luanda,AO,-8.84,13.23,75.42,66,1,14.05,
149,Nokaneng,BW,-19.67,22.27,79.16,17,0,10.16,


* Set parameters to search for hotels with 5000 meters.

In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

* Hit the Google Places API for each city's coordinates.

In [11]:
print('Beginning Hotel Data Retrieval')
print('------------------------------')

for index, row in hotel_df.iterrows():
    # get city name location from hotel df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed

print('------------------------------')
print(f"End of City Data Retrieval for Hotels near our select cities")
print('------------------------------')

Beginning Hotel Data Retrieval
------------------------------
Retrieving Results for Index 5: Vila.
Closest hotel in Vila is Hotel Castrum Villae.
------------
Retrieving Results for Index 22: Blagoveshchenka.
Closest hotel in Blagoveshchenka is " Solienyie oziora" Altai.
------------
Retrieving Results for Index 27: East London.
Closest hotel in East London is Kennaway Hotel.
------------
Retrieving Results for Index 33: Mossamedes.
Closest hotel in Mossamedes is iu Hotel Namibe.
------------
Retrieving Results for Index 56: Onega.
Closest hotel in Onega is Gostevoy Dom "Chayka".
------------
Retrieving Results for Index 66: Saint George.
Closest hotel in Saint George is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 89: Samfya.
Closest hotel in Samfya is Samfya Water front Lodge.
------------
Retrieving Results for Index 109: Richards Bay.
Closest hotel in Richards Bay is Sun 1 Hotel.
------------
Retrieving Results for Index 146: Luanda.
Closest hotel in Luan

* Store the first Hotel result into the DataFrame.

In [12]:
hotel_df

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
5,Vila,PT,42.03,-8.16,84.58,45,0,3.36,Hotel Castrum Villae
22,Blagoveshchenka,RU,52.83,79.87,82.83,46,0,8.59,""" Solienyie oziora"" Altai"
27,East London,ZA,-33.02,27.91,77.00,29,0,5.82,Kennaway Hotel
33,Mossamedes,AO,-15.20,12.15,81.73,29,1,11.45,iu Hotel Namibe
56,Onega,RU,63.91,38.14,76.10,37,0,2.33,"Gostevoy Dom ""Chayka"""
66,Saint George,US,37.10,-113.58,79.25,19,1,5.82,Best Western Plus Abbey Inn
89,Samfya,ZM,-11.36,29.56,76.86,29,0,4.27,Samfya Water front Lodge
109,Richards Bay,ZA,-28.78,32.04,76.59,36,0,11.50,Sun 1 Hotel
146,Luanda,AO,-8.84,13.23,75.42,66,1,14.05,EPIC SANA Luanda
149,Nokaneng,BW,-19.67,22.27,79.16,17,0,10.16,


* Plot markers on top of the heatmap.

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# adding marker layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='388px', margin='0 auto 0 auto', padding='1px', wi…